In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
url = 'https://www.vivareal.com.br/venda/bahia/salvador/?pagina={}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML+, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [179]:
def buscarCasas():
    i = 1
    lst_casas = []

    while True:
        req = requests.get(url.format(i), headers=headers)
        soup = bs(req.text)

        print(f'Processando a página {i}')

        botaoProximo = soup.find('button', {'title': 'Próxima página'})

        casas = soup.find_all('a', {'class': 'property-card__content-link'})
        for casa in casas:
            lst_casas.append(casa)
        i = i+1

        if ('data-disabled' in botaoProximo.attrs):
            return lst_casas

In [180]:
def quantidadeCasas():
    req = requests.get(url.format(1), headers=headers)
    soup = bs(req.text)
    qtd_casas = soup.find(
        'strong', {'class': 'results-summary__count'}).text.replace('.', '')

    return qtd_casas

In [181]:
df = pd.DataFrame(
    columns=[
        'descricao', 'area', 'quartos', 'banheiros', 'vagas', 'valor', 'condominio', 'link'
    ]
)
qtd_casas = int(quantidadeCasas())
print(f'A busca encontrou {qtd_casas}, logo, {int(qtd_casas/36)} páginas')

casas = buscarCasas()

A busca encontrou 77742, logo, 2159 páginas
Processando a página 275
Processando a página 276
Processando a página 277


In [182]:
for casa in casas:
    try:
        descricao = casa.find(
            'span', {'class': 'property-card__title'}).text.strip()
    except:
        descricao = None
    try:
        area = casa.find(
            'li', {'class': 'property-card__detail-area'}).span.text.strip()
    except:
        area = None
    try:
        quartos = casa.find(
            'li', {'class': 'property-card__detail-room'}).span.text.strip()
    except:
        quartos = None
    try:
        banheiros = casa.find(
            'li', {'class': 'property-card__detail-bathroom'}).span.text.strip()
    except:
        banheiros = None
    try:
        vagas = casa.find(
            'li', {'class': 'property-card__detail-garage'}).span.text.strip()
    except:
        vagas = None
    try:
        valor = casa.find(
            'div', {'class': 'property-card__price'}).text.strip()
    except:
        valor = None
    try:
        condominio = casa.find(
            'strong', {'class': 'js-condo-price'}).text.strip()
    except:
        condominio = None
    try:
        link = f"https://www.vivareal.com.br/{casa['href']}"
    except:
        link = None

    df.loc[df.shape[0]] = [descricao, area, quartos,
                           banheiros, vagas, valor, condominio, link]

In [184]:
df.shape[0]

108